In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()

CREATING NEW EMPLOYEE TABLE

In [3]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
employees_data = [(1,"James","Smith","smith@gmail.com","67867567","","1","Developer"),
                 (2,"Michael","Rose","Rose@gmail.com","65768789","","2","Designer"),
                 (3,"Robert","Williams","Robert@gmail.com","787678","","3","Editor"),
                 (4,"Maria","Brown","Brown@gmail.com","65767812","","4","Manager")]

In [4]:
schema = StructType([
    StructField("employee_id",IntegerType(),True),
    StructField("firstname",StringType(),True),
    StructField("lastename",StringType(),True),
    StructField("email",StringType(),True),
    StructField("phone",StringType(),True),
    StructField("hiredate",StringType(),True),
    StructField("manager_id",StringType(),True),
    StructField("job_title",StringType(),True),
])

In [5]:
employees = spark.createDataFrame(data = employees_data,schema = schema)

In [6]:
employees.printSchema()
employees.show(truncate=False)

root
 |-- employee_id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastename: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- manager_id: string (nullable = true)
 |-- job_title: string (nullable = true)

+-----------+---------+---------+----------------+--------+--------+----------+---------+
|employee_id|firstname|lastename|email           |phone   |hiredate|manager_id|job_title|
+-----------+---------+---------+----------------+--------+--------+----------+---------+
|1          |James    |Smith    |smith@gmail.com |67867567|        |1         |Developer|
|2          |Michael  |Rose     |Rose@gmail.com  |65768789|        |2         |Designer |
|3          |Robert   |Williams |Robert@gmail.com|787678  |        |3         |Editor   |
|4          |Maria    |Brown    |Brown@gmail.com |65767812|        |4         |Manager  |
+-----------+---------+---------+--------

In [7]:
Order_data = [(1,8,"in progress",2,"2021"),
             (2,9,"pending",3,"2021"),
             (3,11,"arrived",2,"2021"),
             (4,12,"pending",5,"2021"),
             ]

In [8]:
schema = StructType([
    StructField("order_id",IntegerType(),True),
    StructField("customer_id",StringType(),True),
    StructField("status",StringType(),True),
    StructField("salesman_id",IntegerType(),True),
    StructField("order_date",StringType(),True),
    
])

In [9]:
orders=spark.createDataFrame(data = Order_data,schema=schema)
orders.printSchema()
orders.show(truncate = False)

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- salesman_id: integer (nullable = true)
 |-- order_date: string (nullable = true)

+--------+-----------+-----------+-----------+----------+
|order_id|customer_id|status     |salesman_id|order_date|
+--------+-----------+-----------+-----------+----------+
|1       |8          |in progress|2          |2021      |
|2       |9          |pending    |3          |2021      |
|3       |11         |arrived    |2          |2021      |
|4       |12         |pending    |5          |2021      |
+--------+-----------+-----------+-----------+----------+



In [10]:
Order1_data = [(1,8,"in progress",2,"2021"),
             (2,9,"pending",3,"2021"),
             (3,11,"arrived",2,"2021"),
             (4,12,"pending",5,"2021"),
             ]

In [11]:
schema = StructType([
    StructField("employee_id",IntegerType(),True),
    StructField("customer_id",StringType(),True),
    StructField("status",StringType(),True),
    StructField("salesman_id",IntegerType(),True),
    StructField("order_date",StringType(),True),
    
])

In [12]:
order1=spark.createDataFrame(data = Order_data,schema=schema)
order1.printSchema()
order1.show(truncate = False)

root
 |-- employee_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- salesman_id: integer (nullable = true)
 |-- order_date: string (nullable = true)

+-----------+-----------+-----------+-----------+----------+
|employee_id|customer_id|status     |salesman_id|order_date|
+-----------+-----------+-----------+-----------+----------+
|1          |8          |in progress|2          |2021      |
|2          |9          |pending    |3          |2021      |
|3          |11         |arrived    |2          |2021      |
|4          |12         |pending    |5          |2021      |
+-----------+-----------+-----------+-----------+----------+



Which employees could convince customers to order products?

LEFT SEMI JOIN

In [13]:
salesman = employees.join(orders, f.expr("employee_id == salesman_id"),"left_semi")
salesman.show()

+-----------+---------+---------+----------------+--------+--------+----------+---------+
|employee_id|firstname|lastename|           email|   phone|hiredate|manager_id|job_title|
+-----------+---------+---------+----------------+--------+--------+----------+---------+
|          3|   Robert| Williams|Robert@gmail.com|  787678|        |         3|   Editor|
|          2|  Michael|     Rose|  Rose@gmail.com|65768789|        |         2| Designer|
+-----------+---------+---------+----------------+--------+--------+----------+---------+



In [14]:
employees.createOrReplaceTempView("employees")
orders.createOrReplaceTempView("orders")

In [15]:
spark.sql("select * from employees e left semi join orders o on e.employee_id = o.salesman_id ").show()

+-----------+---------+---------+----------------+--------+--------+----------+---------+
|employee_id|firstname|lastename|           email|   phone|hiredate|manager_id|job_title|
+-----------+---------+---------+----------------+--------+--------+----------+---------+
|          3|   Robert| Williams|Robert@gmail.com|  787678|        |         3|   Editor|
|          2|  Michael|     Rose|  Rose@gmail.com|65768789|        |         2| Designer|
+-----------+---------+---------+----------------+--------+--------+----------+---------+



In [16]:
spark.sql("select employee_id,count(salesman_id) as sales from employees e inner join orders o on employee_id = salesman_id group by e.employee_id").show()

+-----------+-----+
|employee_id|sales|
+-----------+-----+
|          3|    1|
|          2|    2|
+-----------+-----+



In [17]:
sale = employees.join(orders,f.expr("employee_id == salesman_id"),"inner").groupBy("employee_id").agg(count("*").alias("sales")).orderBy("sales",ascending = False)


In [18]:
sale.show()

+-----------+-----+
|employee_id|sales|
+-----------+-----+
|          2|    2|
|          3|    1|
+-----------+-----+



NATURAL JOIN---------

In [19]:
natural_join = employees.join(order1,["employee_id"])
natural_join.show()

+-----------+---------+---------+----------------+--------+--------+----------+---------+-----------+-----------+-----------+----------+
|employee_id|firstname|lastename|           email|   phone|hiredate|manager_id|job_title|customer_id|     status|salesman_id|order_date|
+-----------+---------+---------+----------------+--------+--------+----------+---------+-----------+-----------+-----------+----------+
|          1|    James|    Smith| smith@gmail.com|67867567|        |         1|Developer|          8|in progress|          2|      2021|
|          3|   Robert| Williams|Robert@gmail.com|  787678|        |         3|   Editor|         11|    arrived|          2|      2021|
|          4|    Maria|    Brown| Brown@gmail.com|65767812|        |         4|  Manager|         12|    pending|          5|      2021|
|          2|  Michael|     Rose|  Rose@gmail.com|65768789|        |         2| Designer|          9|    pending|          3|      2021|
+-----------+---------+---------+--------

INNER JOIN EXAMPLE WITH SYNTEX

In [20]:
#inner_join = table1.join(table2,join_specification,join_type)
inner_join = employees.join(orders,(employees["employee_id"] == orders["salesman_id"]),"inner")
inner_join.show()

+-----------+---------+---------+----------------+--------+--------+----------+---------+--------+-----------+-----------+-----------+----------+
|employee_id|firstname|lastename|           email|   phone|hiredate|manager_id|job_title|order_id|customer_id|     status|salesman_id|order_date|
+-----------+---------+---------+----------------+--------+--------+----------+---------+--------+-----------+-----------+-----------+----------+
|          3|   Robert| Williams|Robert@gmail.com|  787678|        |         3|   Editor|       2|          9|    pending|          3|      2021|
|          2|  Michael|     Rose|  Rose@gmail.com|65768789|        |         2| Designer|       1|          8|in progress|          2|      2021|
|          2|  Michael|     Rose|  Rose@gmail.com|65768789|        |         2| Designer|       3|         11|    arrived|          2|      2021|
+-----------+---------+---------+----------------+--------+--------+----------+---------+--------+-----------+-----------+--

CROSS_JOIN

In [21]:
cross_join = employees.crossJoin(orders)
cross_join.show()

+-----------+---------+---------+----------------+--------+--------+----------+---------+--------+-----------+-----------+-----------+----------+
|employee_id|firstname|lastename|           email|   phone|hiredate|manager_id|job_title|order_id|customer_id|     status|salesman_id|order_date|
+-----------+---------+---------+----------------+--------+--------+----------+---------+--------+-----------+-----------+-----------+----------+
|          1|    James|    Smith| smith@gmail.com|67867567|        |         1|Developer|       1|          8|in progress|          2|      2021|
|          1|    James|    Smith| smith@gmail.com|67867567|        |         1|Developer|       2|          9|    pending|          3|      2021|
|          1|    James|    Smith| smith@gmail.com|67867567|        |         1|Developer|       3|         11|    arrived|          2|      2021|
|          1|    James|    Smith| smith@gmail.com|67867567|        |         1|Developer|       4|         12|    pending|  

Use the RDD API to identify how many orders each customer is waiting for (this means, status = "Pending"). 
The customers are identified only by customer id. Sort the resulting RDD by the number of pending orders in descending order. 
The variable orders is a reference to the DataFrame Orders.

 

In [32]:
orders.rdd.filter(lambda x:x["status"]=="pending").map(lambda x:(x["customer_id"],1)).reduceByKey(lambda x,y : x+y).sortBy(lambda x:x[1],ascending=False).collect()

[('12', 1), ('9', 1)]